# 02 pQTL UKBB processing

**Origin:** `0_2_processing_pqtl_ukbb.ipynb`  
**This annotated version was generated on:** 2025-10-13 06:41

**What this notebook does (high level):**  
- Process UK Biobank pQTL summary data: standardize formats, map to proteins/genes, and filter by QC thresholds.

**How to use:**  
1. Review the markdown notes before each code cell.  
2. Adjust input/output paths as needed for your environment.  
3. Run cell-by-cell to reproduce artifacts for downstream steps.

---


**Step 1:** Load tabular data (summary stats / annotations).

In [51]:




path2dhs = "/mnt/f/0.data/dhs/DHS_Index_and_Vocabulary_hg38_WM20190703.txt.gz"
dhsindex = pd.read_csv( path2dhs , sep = '\t' )

import pandas as pd
import pybedtools
import gzip
import os
from concurrent.futures import ProcessPoolExecutor

vcf_dir = "/mnt/f/0.data/ens_vcf/"
dhs_path = "/mnt/f/0.data/dhs/DHS_Index_and_Vocabulary_hg38_WM20190703.txt.gz"
out_dir = "/mnt/f/0.data/ens_vcf_dhs/"
os.makedirs(out_dir, exist_ok=True)

def vcf_chunk_reader(vcf_path, chunk_size=100000):
    buffer = []
    with gzip.open(vcf_path, 'rt') as f:
        for line in f:
            if line.startswith('#'):
                continue
            buffer.append(line.strip().split('\t'))
            if len(buffer) == chunk_size:
                df = pd.DataFrame(buffer, columns=['chr', 'pos', 'rsid', 'ref', 'alt', 'qual', 'filter', 'info'])
                yield df
                buffer = []
        if buffer:
            df = pd.DataFrame(buffer, columns=['chr', 'pos', 'rsid', 'ref', 'alt', 'qual', 'filter', 'info'])
            yield df

def snps_to_bed(df):
    # 'chrom', 'start', 'end', 'rsid', 'POS'
    df['pos'] = df['pos'].astype(int)
    return df.assign(
        start=df['pos'] - 1,
        end=df['pos'],
        POS=df['pos'],
    ).rename(columns={'chr':'chrom'})[['chrom','start','end','rsid','POS']]

def dhs_to_bed(dhs_df, chrom, component):
    # 'chrom', 'start', 'end', 'identifier'
    sub = dhs_df[(dhs_df['Chromosome'] == chrom) & (dhs_df['component'] == component)]
    return sub[['Chromosome', 'Start', 'End', 'identifier']].rename(
        columns={'Chromosome':'chrom','Start':'start','End':'end'}
    )

def process_chrom(chrom):
    chr_str = str(chrom)
    vcf_path = os.path.join(vcf_dir, f"homo_sapiens-chr{chrom}.vcf.gz")
    out_path = os.path.join(out_dir, f"rsid_in_DHS_chr{chrom}.tsv")
    out_path_lym = os.path.join(out_dir, f"rsid_in_DHS_chr{chrom}_lym.tsv")
    out_path_mye = os.path.join(out_dir, f"rsid_in_DHS_chr{chrom}_mye.tsv")
    
    # DHS BEDs for the chromosome
    lym_bed = pybedtools.BedTool.from_dataframe(dhs_to_bed(dhs, chr_str, 'Lymphoid'))
    mye_bed = pybedtools.BedTool.from_dataframe(dhs_to_bed(dhs, chr_str, 'Myeloid / erythroid'))
    
    results_lym, results_mye = [], []
    
    for chunk in vcf_chunk_reader(vcf_path, chunk_size=100000):
        if chunk.empty:
            continue
        snps_bed = pybedtools.BedTool.from_dataframe(snps_to_bed(chunk))
        # Intersect with DHS, -wa -wb: report all columns from A (SNP) and B (DHS)
        lym_overlap = snps_bed.intersect(lym_bed, wa=True, wb=True)
        mye_overlap = snps_bed.intersect(mye_bed, wa=True, wb=True)
        
        if len(lym_overlap) > 0:
            # Columns: snp bed: chrom, start, end, rsid, POS; dhs bed: chrom, start, end, identifier
            df_lym = lym_overlap.to_dataframe(names=[
                'chr', 'start', 'end', 'rsid', 'POS', 'dhs_chr', 'dhs_start', 'dhs_end', 'identifier'
            ])
            df_lym['component'] = 'Lymphoid'
            results_lym.append(df_lym[['rsid', 'POS', 'component', 'identifier']])
        if len(mye_overlap) > 0:
            df_mye = mye_overlap.to_dataframe(names=[
                'chr', 'start', 'end', 'rsid', 'POS', 'dhs_chr', 'dhs_start', 'dhs_end', 'identifier'
            ])
            df_mye['component'] = 'Mye_ery'
            results_mye.append(df_mye[['rsid', 'POS', 'component', 'identifier']])
    
    df_lym = pd.concat(results_lym, ignore_index=True) if results_lym else pd.DataFrame(columns=['rsid', 'POS', 'component', 'identifier'])
    df_mye = pd.concat(results_mye, ignore_index=True) if results_mye else pd.DataFrame(columns=['rsid', 'POS', 'component', 'identifier'])
    df_lym.to_csv(out_path, sep='\t', index=False)
    df_lym.to_csv(out_path, sep='\t', index=False)
    if 1 :
        df_all = pd.concat([df_lym, df_mye], ignore_index=True)
        # Merge: for rsid+POS+identifier found in both, assign 'Lym_Mye_ery'
        if not df_all.empty:
            # group by rsid+POS+identifier, assign merged component
            df_all = df_all.groupby(['rsid','POS','identifier'])['component'].agg(
                lambda x: 'Lym_Mye_ery' if set(x) == {'Lymphoid','Mye_ery'} else list(x)[0]
            ).reset_index()
            df_all.to_csv(out_path, sep='\t', index=False)
            print(f"{out_path} written: {len(df_all)} SNPs.")
        else:
            print(f"{out_path}: No SNPs found.")

# --- Load DHS only once for all workers (top level, so it's pickled for each subprocess)
dhs = pd.read_csv(dhs_path, sep='\t')
dhs['identifier'] = dhs['identifier'].astype(str)
dhs['Chromosome'] = dhs['seqname'].str.replace('chr', '')
dhs['Start'] = dhs['start']
dhs['End'] = dhs['end']

# --- Run with 6 workers
if __name__ == "__main__":
    with ProcessPoolExecutor(max_workers=6) as executor:
        list(executor.map(process_chrom, range(1, 23)))



import pandas as pd
import glob

# 1. Collect all rsids in DHS (all chromosomes)
dhs_dir = "/mnt/f/0.data/ens_vcf_dhs/"
lym_files = glob.glob(dhs_dir + "rsid_in_DHS_chr*_lym.tsv")
mye_files = glob.glob(dhs_dir + "rsid_in_DHS_chr*_mye.tsv")
lymph_rsids = set() 
myeloid_rsids = set()

    
for f in lym_files:
    tmp = pd.read_csv(f, sep='\t', dtype=str)    
    lymph_rsids.update( tmp.loc[ :,'rsid'])
    
for f in mye_files:
    tmp = pd.read_csv(f, sep='\t', dtype=str)    
    myeloid_rsids.update( tmp.loc[ :,'rsid'])
    






**Step 2:** Load tabular data (summary stats / annotations).

In [ ]:

import pandas as pd
import os
import glob

dhs_dir = "/mnt/f/0.data/ens_vcf_dhs/"
lym_files = glob.glob(os.path.join(dhs_dir, "rsid_in_DHS_chr*_lym.tsv"))
mye_files = glob.glob(os.path.join(dhs_dir, "rsid_in_DHS_chr*_mye.tsv"))

# Efficient rsid set building
def load_rsid_set(files):
    rsid_set = set()
    for f in files:
        for rsid in pd.read_csv(f, sep='\t', usecols=['rsid'], dtype=str, chunksize=200_000):
            rsid_set.update(rsid['rsid'].tolist())
    return rsid_set & outcome_rsid

"""
Collecting lymphoid rsids...
  Lymphoid: 19,578,469 unique rsids
Collecting myeloid rsids...
  Myeloid: 13,010,252 unique rsids
"""

print("Collecting lymphoid rsids...")
lymph_rsids = load_rsid_set(lym_files)
print(f"  Lymphoid: {len(lymph_rsids):,} unique rsids")
print("Collecting myeloid rsids...")
myeloid_rsids = load_rsid_set(mye_files)
print(f"  Myeloid: {len(myeloid_rsids):,} unique rsids")








**Step 3:** Load tabular data (summary stats / annotations).

In [ ]:



import os
import tarfile
import pandas as pd
from pybedtools import BedTool
import shutil

# Paths
pqtl_dir = "/mnt/e/3.thrombocytopenia/pqtl/UKB-PPP pGWAS summary statistics/European (discovery)"
dhs_path = "/mnt/f/0.data/dhs/DHS_Index_and_Vocabulary_hg38_WM20190703.txt.gz"
output_dir = "/mnt/f/0.data/ukb_ppp/"
os.makedirs(output_dir, exist_ok=True)

# 1. Load and filter DHS index
dhs = pd.read_csv(dhs_path, sep='\t')
mask = dhs['component'].isin(['Lymphoid', 'Myeloid / erythroid'])
dhs_sub = dhs[mask].copy()
dhs_bed = BedTool.from_dataframe(
    #dhs_sub[['seqname', 'start', 'end']].rename(columns={'seqname': 'chrom'})
    #dhs_bed = BedTool.from_dataframe(
    dhs_sub[['seqname', 'start', 'end', 'identifier']].rename(columns={'seqname': 'chrom'})
)


# 2. Function to process one chromosome pQTL file
def process_pqtl_file(pqtl_file, dhs_bed):
    df = pd.read_csv(pqtl_file, sep=' ')
    # 1. P < 5e-8 filter
    df = df[df['LOG10P'].astype(float) > 7.30103].copy()  # -log10(5e-8) = 7.30103
    if df.empty:
        return pd.DataFrame()
    # 2. DHS overlap
    df['chrom'] = 'chr' + df['CHROM'].astype(str)
    df['start'] = df['GENPOS'] - 1
    df['end'] = df['GENPOS']
    pqtl_bed = BedTool.from_dataframe(df[['chrom', 'start', 'end', 'ID']])
    # Intersect with DHS, outputting both sides
    overlap = pqtl_bed.intersect(dhs_bed, wa=True, wb=True)
    # Build output with DHS identifier
    records = []
    for feature in overlap:
        pqtl_id = feature[3]  # ID from pqtl
        dhs_identifier = feature[7]  # 'identifier' from DHS
        pqtl_row = df[df['ID'] == pqtl_id].iloc[0].to_dict()
        pqtl_row['DHS_identifier'] = dhs_identifier
        records.append(pqtl_row)
    if records:
        return pd.DataFrame(records)
    else:
        return pd.DataFrame()







**Step 4:** Join/merge datasets to align keys across resources.

In [ ]:

def process_tarball(fname):
    import os, tarfile, pandas as pd, shutil
    from pybedtools import BedTool

    # Use globals or pass as function args
    pqtl_dir = "/mnt/e/3.thrombocytopenia/pqtl/UKB-PPP pGWAS summary statistics/European (discovery)"
    output_dir = "/mnt/f/0.datasets/dhs_context/ukb_ppp/"
    dhs_bed = process_tarball.dhs_bed  # attach as static var below

    tarball_path = os.path.join(pqtl_dir, fname)
    basename = fname.replace('.tar','')
    out_prefix = os.path.join(output_dir, basename)
    tmp_extract_dir = os.path.join(output_dir, 'tmp_' + basename)
    os.makedirs(tmp_extract_dir, exist_ok=True)
    # Untar
    with tarfile.open(tarball_path) as tar:
        tar.extractall(tmp_extract_dir)
    # Find relevant chromosome files
    tmp_extract_dir = tmp_extract_dir + '/' + basename 
    pqtl_files = [os.path.join(tmp_extract_dir, f) for f in os.listdir( tmp_extract_dir ) 
                  if f.startswith('discovery_chr') and not f.startswith('discovery_chrX')]
    results = []
    for pqtl_file in pqtl_files:
        result = process_pqtl_file(pqtl_file, dhs_bed)
        if not result.empty:
            results.append(result)
    if results:
        all_filtered = pd.concat(results, ignore_index=True)
        print( basename, all_filtered.shape )
        # Write to output, only if at least one result
        out_path = out_prefix + '_dhs.txt.gz'
        all_filtered.to_csv(out_path, sep='\t', index=False, compression='gzip')
    # Clean up temp files
    shutil.rmtree(tmp_extract_dir)






print("Processing complete!")






**Step 5:** Run a processing or analysis step.

In [52]:
all_tarballs = [f for f in os.listdir(pqtl_dir) if f.endswith('.tar')]

process_tarball.dhs_bed = dhs_bed  # outside the function, after dhs_bed is created


import concurrent.futures

all_tarballs = all_tarballs[474:] 

max_workers = 5  # or os.cpu_count(), or fewer for IO-bound
with concurrent.futures.ProcessPoolExecutor(max_workers=max_workers) as executor:
    # Optionally: use tqdm for progress bar
    list(executor.map(process_tarball, all_tarballs))




CDAN1_Q8IWY9_OID31076_v1_Neurology_II (18, 18)
CDA_P32320_OID30345_v1_Cardiometabolic_II (397, 18)
CDC26_Q8NHZ8_OID31508_v1_Oncology_II (49, 18)
CDC27_P30260_OID21238_v1_Oncology (14, 18)
CDC37_Q16543_OID21470_v1_Oncology (49, 18)
CDC42BPB_Q9Y5S2_OID31485_v1_Oncology_II (14, 18)
CDH15_P55291_OID21005_v1_Neurology (238, 18)
CDCP1_Q9H5V8_OID20940_v1_Neurology (1354, 18)
CDH17_Q12864_OID20134_v1_Cardiometabolic (1531, 18)
CDH1_P12830_OID20353_v1_Cardiometabolic (159, 18)
CDH23_Q9H251_OID31074_v1_Neurology_II (23, 18)
CDH2_P19022_OID20170_v1_Cardiometabolic (82, 18)
CDH3_P22223_OID21008_v1_Neurology (111, 18)
CDH5_P33151_OID20243_v1_Cardiometabolic (185, 18)
CDH6_P55285_OID20157_v1_Cardiometabolic (57, 18)
CDHR1_Q96JP9_OID20808_v1_Neurology (51, 18)
CDHR2_Q9BYE9_OID21282_v1_Oncology (48, 18)
CDHR5_Q9HBB8_OID20193_v1_Cardiometabolic (323, 18)
CDK1_P06493_OID31164_v1_Oncology_II (3, 18)
CDK5RAP3_Q96JB5_OID30985_v1_Neurology_II (37, 18)
CDKN1A_P38936_OID21319_v1_Oncology (56, 18)
CDKN2D_P5527

**Step 6:** Run a processing or analysis step.

**Step 7:** Load tabular data (summary stats / annotations).

In [1]:

import os, glob, re
import pandas as pd
import numpy as np
from pathlib import Path

# ---- Inputs ----
pqtl_dir = '/mnt/f/10_osteo_MR/datasets/dhs_context/ukb_ppp/'  # per-gene pQTL files: <GENE>_dhs.txt.gz
rsid_map_dir = '/mnt/e/0.datasets/pqtl_ukb_ppp/Metadata/SNP RSID maps/'  # per-chr ID→rsid maps
outcome_with_dhs = "/mnt/f/10_osteo_MR/MR_ready/outcome_osteo_within_wb_DHS.tsv"  # columns: SNP (may be 'rs1;rs2;...'), dhs_id

# ---- Output ----
output_file = '/mnt/f/10_osteo_MR/MR_ready/exposure_ukbppp_pqtl_dhs_index.tsv'
Path(os.path.dirname(output_file)).mkdir(parents=True, exist_ok=True)

# ---------- 0) RSIDs within DHS (and dhs_id) from outcome ----------
splitter = re.compile(r"[;,\s]+")
snp2dhs = {}
out_df = pd.read_csv(outcome_with_dhs, sep="\t", dtype="string", usecols=["SNP","dhs_id"])
for s, did in zip(out_df["SNP"], out_df["dhs_id"]):
    if pd.isna(s): 
        continue
    did_str = "" if pd.isna(did) else str(did)
    for tok in splitter.split(str(s)):
        if tok and tok != "." and tok not in snp2dhs:
            snp2dhs[tok] = did_str
rsids_in_dhs = set(snp2dhs.keys())
print(f"[info] RSIDs from outcome within DHS: {len(rsids_in_dhs):,}")

# ---------- 1) Merge all per-gene pQTL files ----------
pqtl_files = sorted(glob.glob(os.path.join(pqtl_dir, '*_dhs.txt.gz')))
if not pqtl_files:
    raise FileNotFoundError(f"No files matching '*_dhs.txt.gz' in {pqtl_dir}")

all_pqtl = []
for pqtl_f in pqtl_files:
    gene = os.path.basename(pqtl_f).split('_')[0]
    df = pd.read_csv(pqtl_f, sep='\t', compression='gzip', dtype='string')
    df['gene'] = gene
    all_pqtl.append(df)

merged = pd.concat(all_pqtl, ignore_index=True)
print(f"[merge] total rows: {merged.shape[0]:,}")

# ---------- 2) Map internal variant IDs -> rsid ----------
unique_ids = set(merged['ID'].dropna().unique())
print(f"[map] unique internal IDs: {len(unique_ids):,}")

id2rsid = {}
for chrom in range(1, 23):
    map_path = os.path.join(rsid_map_dir, f'olink_rsid_map_mac5_info03_b0_7_chr{chrom}_patched_v2.tsv.gz')
    if not os.path.exists(map_path):
        print(f"[warn] Missing RSID map for chromosome {chrom}")
        continue
    dfm = pd.read_csv(map_path, sep='\t', usecols=['ID','rsid'], dtype='string', compression='infer')
    dfm = dfm[dfm['ID'].isin(unique_ids)]
    if not dfm.empty:
        id2rsid.update(dfm.set_index('ID')['rsid'].to_dict())

print(f"[map] total IDs mapped to rsid: {len(id2rsid):,}")

merged['rsid'] = merged['ID'].map(id2rsid)
merged = merged[merged['rsid'].notna() & (merged['rsid'] != "")].copy()
print(f"[filter] rows with mappable rsid: {len(merged):,}")

# ---------- 3) Keep only RSIDs that are in outcome DHS set; attach dhs_id ----------
merged = merged[merged['rsid'].isin(rsids_in_dhs)].copy()
merged['dhs_id'] = merged['rsid'].map(snp2dhs).astype('string')
print(f"[filter] rows after outcome-DHS filter: {len(merged):,}")

# ---------- 4) Prepare stats; compute pval and |Z| ----------
# Numeric conversions (safe)
merged['BETA']   = pd.to_numeric(merged['BETA'], errors='coerce')
merged['SE']     = pd.to_numeric(merged['SE'], errors='coerce')
merged['LOG10P'] = pd.to_numeric(merged['LOG10P'], errors='coerce')

merged['pval'] = np.power(10.0, -merged['LOG10P'])
merged['Zabs'] = (merged['BETA'] / merged['SE']).abs()
merged.loc[~np.isfinite(merged['Zabs']), 'Zabs'] = np.nan

# ---------- 5) One SNP per (gene × dhs_id): min pval, tie by max |Z|, then first ----------
merged = merged.dropna(subset=['dhs_id','gene'])  # ensure groups are valid
merged = merged.sort_values(['gene','dhs_id','pval','Zabs'], ascending=[True, True, True, False])
best = merged.groupby(['gene','dhs_id'], as_index=False).head(1).copy()
print(f"[select] final (gene × DHS) picks: {len(best):,}")

# ---------- 6) Build MR-ready table ----------
mr_df = pd.DataFrame({
    'SNP': best['rsid'],
    'effect_allele': best['ALLELE1'],  # ALLELE1 = effect (alt) in these pQTLs
    'other_allele':  best['ALLELE0'],
    'beta': best['BETA'],
    'se':   best['SE'],
    'pval': best['pval'],
    'gene': best['gene'],
    'dhs_id': best['dhs_id'],          # keep for traceability
    # Optional extras (uncomment if you want them preserved)
    # 'CHROM': best.get('CHROM', pd.NA),
    # 'GENPOS': best.get('GENPOS', pd.NA),
    # 'N': best.get('N', pd.NA),
})

mr_df.to_csv(output_file, sep='\t', index=False)
print(f"[done] MR exposure file written: {output_file}  (rows: {len(mr_df):,})")





[info] RSIDs from outcome within DHS: 606,769
[merge] total rows: 633,569
[map] unique internal IDs: 85,030
[map] total IDs mapped to rsid: 85,030
[filter] rows with mappable rsid: 633,569
[filter] rows after outcome-DHS filter: 427,601
[select] final (gene × DHS) picks: 226,718
[done] MR exposure file written: /mnt/f/10_osteo_MR/MR_ready/exposure_ukbppp_pqtl_dhs_index.tsv  (rows: 226,718)
